In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn import preprocessing

In [3]:
from sklearn.metrics import precision_score

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

In [5]:
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold


from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import RFE

from sklearn import metrics
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import classification_report,confusion_matrix
import sklearn.model_selection as model_selection

from random import sample

from sklearn import preprocessing

from sklearn.model_selection import validation_curve
from sklearn.pipeline import make_pipeline
from sklearn.metrics import make_scorer

from sklearn.metrics import accuracy_score

from sklearn.pipeline import Pipeline

from sklearn.model_selection import GridSearchCV


### Column Description

● store_id - The unique identifier of a store.

● product_id - The unique identifier of a product.

● date - Sales date (YYYY-MM-DD)

● sales - Sales quantity

● revenue - Daily total sales revenue

● stock - End of day stock quantity

● price - Product sales price

● promo_type_1 - Type of promotion applied on channel 1

● promo_bin_1 - Binned promotion rate for applied promo_type_1

● promo_type_2 - Type of promotion applied on channel 2

● promo_bin_2 - Binned promotion rate for applied promo_type_2

● promo_discount_2 - Discount rate for applied promo type 2

● promo_discount_type_2 - Type of discount applied

● product_length - Length of product

● product_depth - Depth of product

● product_width - Width of product

● hierarchy1_id

● hierarchy2_id

● hierarchy3_id

● hierarchy4_id

● hierarchy5_id

● storetype_id

● store_size

● city_id

● train_or_test - rows with train tag will be used to train KNNRegressor and rows with test
tag will be used for accuracy calculation

## Code

In [386]:
sales = pd.read_csv("sales.csv")
ph = pd.read_csv("product_hierarchy.csv")
store_cities = pd.read_csv("store_cities.csv")

In [387]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [388]:
print("Sales size: ", sales.shape)
print("Product Hierarchy size: ", ph.shape)
print("Store Cities size: ", store_cities.shape)

Sales size:  (19454838, 13)
Product Hierarchy size:  (699, 10)
Store Cities size:  (144, 4)


In [389]:
sales.head()

,product_id,store_id,date,sales,revenue,stock,price,promo_type_1,promo_bin_1,promo_type_2,promo_bin_2,promo_discount_2,promo_discount_type_2
0,P0001,S0002,2017-01-02,0.0,0.00,8.0,6.25,PR14,NaN,PR03,NaN,NaN,NaN
1,P0001,S0012,2017-01-02,1.0,5.30,0.0,6.25,PR14,NaN,PR03,NaN,NaN,NaN
2,P0001,S0013,2017-01-02,2.0,10.59,0.0,6.25,PR14,NaN,PR03,NaN,NaN,NaN
3,P0001,S0023,2017-01-02,0.0,0.00,6.0,6.25,PR14,NaN,PR03,NaN,NaN,NaN
4,P0001,S0025,2017-01-02,0.0,0.00,1.0,6.25,PR14,NaN,PR03,NaN,NaN,NaN


In [390]:
sales.isna().sum()

product_id                      0
store_id                        0
date                            0
sales                     1190762
revenue                   1190762
stock                     1190762
price                      571981
promo_type_1                    0
promo_bin_1              16803149
promo_type_2                    0
promo_bin_2              19395757
promo_discount_2         19395757
promo_discount_type_2    19395757
dtype: int64

In [391]:
print("Min Date: ", sales.date.min() )
print("Max Date: ", sales.date.max() )

Min Date:  2017-01-02
Max Date:  2019-12-29


In [392]:
ph.head()

,product_id,product_length,product_depth,product_width,cluster_id,hierarchy1_id,hierarchy2_id,hierarchy3_id,hierarchy4_id,hierarchy5_id
0,P0000,5.0,20.0,12.0,NaN,H00,H0004,H000401,H00040105,H0004010534
1,P0001,13.5,22.0,20.0,cluster_5,H01,H0105,H010501,H01050100,H0105010006
2,P0002,22.0,40.0,22.0,cluster_0,H03,H0315,H031508,H03150800,H0315080028
3,P0004,2.0,13.0,4.0,cluster_3,H03,H0314,H031405,H03140500,H0314050003
4,P0005,16.0,30.0,16.0,cluster_9,H03,H0312,H031211,H03121109,H0312110917


In [393]:
store_cities.head()

,store_id,storetype_id,store_size,city_id
0,S0091,ST04,19,C013
1,S0012,ST04,28,C005
2,S0045,ST04,17,C008
3,S0032,ST03,14,C019
4,S0027,ST04,24,C022


### Q3 - Classification #2

When there is a new product in the system, every business user has a business experience to guess what will be the seasonality curve in the future. Can we replace a business user's experience with an algorithm?

In [394]:
q3_submissions = pd.read_csv("Q3/Q3_submission.csv")

In [395]:
predictions= list(q3_submissions.product_id.values)

In [396]:
sales_products = sales.merge(ph,on='product_id', how='inner')

In [397]:
sales_products.head()

,product_id,store_id,date,sales,revenue,stock,price,promo_type_1,promo_bin_1,promo_type_2,promo_bin_2,promo_discount_2,promo_discount_type_2,product_length,product_depth,product_width,cluster_id,hierarchy1_id,hierarchy2_id,hierarchy3_id,hierarchy4_id,hierarchy5_id
0,P0001,S0002,2017-01-02,0.0,0.00,8.0,6.25,PR14,NaN,PR03,NaN,NaN,NaN,13.5,22.0,20.0,cluster_5,H01,H0105,H010501,H01050100,H0105010006
1,P0001,S0012,2017-01-02,1.0,5.30,0.0,6.25,PR14,NaN,PR03,NaN,NaN,NaN,13.5,22.0,20.0,cluster_5,H01,H0105,H010501,H01050100,H0105010006
2,P0001,S0013,2017-01-02,2.0,10.59,0.0,6.25,PR14,NaN,PR03,NaN,NaN,NaN,13.5,22.0,20.0,cluster_5,H01,H0105,H010501,H01050100,H0105010006
3,P0001,S0023,2017-01-02,0.0,0.00,6.0,6.25,PR14,NaN,PR03,NaN,NaN,NaN,13.5,22.0,20.0,cluster_5,H01,H0105,H010501,H01050100,H0105010006
4,P0001,S0025,2017-01-02,0.0,0.00,1.0,6.25,PR14,NaN,PR03,NaN,NaN,NaN,13.5,22.0,20.0,cluster_5,H01,H0105,H010501,H01050100,H0105010006


In [398]:
sales.shape

(19454838, 13)

In [399]:
sales_products.shape

(19454838, 22)

In [400]:
print("Unique Products in ph: ",len(ph.product_id.unique()))
print("Unique Products in sales: ",len(sales.product_id.unique()))

Unique Products in ph:  699
Unique Products in sales:  649


In [401]:
q3_train = ph[~ph.product_id.isin(predictions)]
q3_submission = ph[ph.product_id.isin(predictions)]

In [402]:
q3_train.head()

,product_id,product_length,product_depth,product_width,cluster_id,hierarchy1_id,hierarchy2_id,hierarchy3_id,hierarchy4_id,hierarchy5_id
1,P0001,13.5,22.0,20.0,cluster_5,H01,H0105,H010501,H01050100,H0105010006
2,P0002,22.0,40.0,22.0,cluster_0,H03,H0315,H031508,H03150800,H0315080028
3,P0004,2.0,13.0,4.0,cluster_3,H03,H0314,H031405,H03140500,H0314050003
4,P0005,16.0,30.0,16.0,cluster_9,H03,H0312,H031211,H03121109,H0312110917
5,P0006,8.5,15.0,15.0,cluster_0,H03,H0316,H031608,H03160817,H0316081708


In [403]:
q3_submission.head()

,product_id,product_length,product_depth,product_width,cluster_id,hierarchy1_id,hierarchy2_id,hierarchy3_id,hierarchy4_id,hierarchy5_id
0,P0000,5.0,20.0,12.0,NaN,H00,H0004,H000401,H00040105,H0004010534
12,P0013,7.1,27.5,13.7,NaN,H01,H0105,H010503,H01050309,H0105030906
22,P0023,1.0,1.0,1.0,NaN,H00,H0004,H000403,H00040305,H0004030536
38,P0040,7.0,26.5,7.0,NaN,H03,H0312,H031204,H03120400,H0312040000
53,P0056,NaN,12.0,10.0,NaN,H03,H0314,H031401,H03140102,H0314010203


In [404]:
q3_train.sort_values(by="cluster_id",inplace=True)
q3_train.index = q3_train.product_id
q3_train.drop("product_id",axis=1, inplace=True)

q3_submission.index = q3_submission.product_id
q3_submission.drop("product_id",axis=1, inplace=True)

In [405]:
q3_train.head()

,product_length,product_depth,product_width,cluster_id,hierarchy1_id,hierarchy2_id,hierarchy3_id,hierarchy4_id,hierarchy5_id
product_id,,,,,,,,,
P0376,2.0,13.0,8.0,cluster_0,H00,H0001,H000105,H00010510,H0001051017
P0470,7.0,18.0,19.0,cluster_0,H00,H0002,H000200,H00020000,H0002000026
P0469,3.8,21.6,35.0,cluster_0,H03,H0313,H031302,H03130210,H0313021001
P0468,5.0,42.0,12.0,cluster_0,H03,H0315,H031502,H03150209,H0315020911
P0467,2.5,17.0,7.0,cluster_0,H03,H0315,H031504,H03150411,H0315041121


In [406]:
q3_submission.head()

,product_length,product_depth,product_width,cluster_id,hierarchy1_id,hierarchy2_id,hierarchy3_id,hierarchy4_id,hierarchy5_id
product_id,,,,,,,,,
P0000,5.0,20.0,12.0,NaN,H00,H0004,H000401,H00040105,H0004010534
P0013,7.1,27.5,13.7,NaN,H01,H0105,H010503,H01050309,H0105030906
P0023,1.0,1.0,1.0,NaN,H00,H0004,H000403,H00040305,H0004030536
P0040,7.0,26.5,7.0,NaN,H03,H0312,H031204,H03120400,H0312040000
P0056,NaN,12.0,10.0,NaN,H03,H0314,H031401,H03140102,H0314010203


In [407]:
q3_train.cluster_id.value_counts()

cluster_0    450
cluster_9     51
cluster_4     36
cluster_3     31
cluster_6     28
cluster_8     14
cluster_5     13
cluster_2     12
cluster_7     10
cluster_1      4
Name: cluster_id, dtype: int64

**Label Encoding**

In [408]:
le = preprocessing.LabelEncoder()

In [409]:
le.fit(q3_train.cluster_id)

LabelEncoder()

In [410]:
q3_train.cluster_id = le.transform(q3_train.cluster_id)

In [411]:
sales_products.head()

,product_id,store_id,date,sales,revenue,stock,price,promo_type_1,promo_bin_1,promo_type_2,promo_bin_2,promo_discount_2,promo_discount_type_2,product_length,product_depth,product_width,cluster_id,hierarchy1_id,hierarchy2_id,hierarchy3_id,hierarchy4_id,hierarchy5_id
0,P0001,S0002,2017-01-02,0.0,0.00,8.0,6.25,PR14,NaN,PR03,NaN,NaN,NaN,13.5,22.0,20.0,cluster_5,H01,H0105,H010501,H01050100,H0105010006
1,P0001,S0012,2017-01-02,1.0,5.30,0.0,6.25,PR14,NaN,PR03,NaN,NaN,NaN,13.5,22.0,20.0,cluster_5,H01,H0105,H010501,H01050100,H0105010006
2,P0001,S0013,2017-01-02,2.0,10.59,0.0,6.25,PR14,NaN,PR03,NaN,NaN,NaN,13.5,22.0,20.0,cluster_5,H01,H0105,H010501,H01050100,H0105010006
3,P0001,S0023,2017-01-02,0.0,0.00,6.0,6.25,PR14,NaN,PR03,NaN,NaN,NaN,13.5,22.0,20.0,cluster_5,H01,H0105,H010501,H01050100,H0105010006
4,P0001,S0025,2017-01-02,0.0,0.00,1.0,6.25,PR14,NaN,PR03,NaN,NaN,NaN,13.5,22.0,20.0,cluster_5,H01,H0105,H010501,H01050100,H0105010006


In [412]:
sales_by_4 = sales_products.groupby("hierarchy4_id").sum()
sales_by_3 = sales_products.groupby("hierarchy3_id").sum()
sales_by_2 = sales_products.groupby("hierarchy2_id").sum()
sales_by_1 = sales_products.groupby("hierarchy1_id").sum()

In [413]:
sales_by_4.drop(sales_by_4.columns[5:],axis=1,inplace=True)
sales_by_3.drop(sales_by_3.columns[5:],axis=1,inplace=True)
sales_by_2.drop(sales_by_2.columns[5:],axis=1,inplace=True)
sales_by_1.drop(sales_by_1.columns[5:],axis=1,inplace=True)

In [414]:
sales_by_4.head()

,sales,revenue,stock,price,promo_discount_2
hierarchy4_id,,,,,
H00000300,960994.0,2.839859e+06,12955868.0,8.700317e+05,45576.0
H00000309,33588.0,1.066802e+05,1099221.0,3.743374e+05,8340.0
H00000310,104069.0,1.664662e+05,2420290.0,1.895040e+05,26248.0
H00000400,140128.0,5.271324e+05,4259082.0,1.376709e+06,29180.0
H00000405,236784.0,8.547498e+05,10302842.0,1.925942e+06,62410.0


In [415]:
q3_train.head()

,product_length,product_depth,product_width,cluster_id,hierarchy1_id,hierarchy2_id,hierarchy3_id,hierarchy4_id,hierarchy5_id
product_id,,,,,,,,,
P0376,2.0,13.0,8.0,0,H00,H0001,H000105,H00010510,H0001051017
P0470,7.0,18.0,19.0,0,H00,H0002,H000200,H00020000,H0002000026
P0469,3.8,21.6,35.0,0,H03,H0313,H031302,H03130210,H0313021001
P0468,5.0,42.0,12.0,0,H03,H0315,H031502,H03150209,H0315020911
P0467,2.5,17.0,7.0,0,H03,H0315,H031504,H03150411,H0315041121


In [416]:
q3_submission.head()

,product_length,product_depth,product_width,cluster_id,hierarchy1_id,hierarchy2_id,hierarchy3_id,hierarchy4_id,hierarchy5_id
product_id,,,,,,,,,
P0000,5.0,20.0,12.0,NaN,H00,H0004,H000401,H00040105,H0004010534
P0013,7.1,27.5,13.7,NaN,H01,H0105,H010503,H01050309,H0105030906
P0023,1.0,1.0,1.0,NaN,H00,H0004,H000403,H00040305,H0004030536
P0040,7.0,26.5,7.0,NaN,H03,H0312,H031204,H03120400,H0312040000
P0056,NaN,12.0,10.0,NaN,H03,H0314,H031401,H03140102,H0314010203


In [417]:
train = q3_train.merge(sales_by_4,how="inner",on="hierarchy4_id")

In [418]:
train = train.merge(sales_by_3,how="inner",on="hierarchy3_id")

In [419]:
train = train.merge(sales_by_2,how="inner",on="hierarchy2_id")

In [420]:
train = train.merge(sales_by_1,how="inner",on="hierarchy1_id")

In [421]:
train.head()

,product_length,product_depth,product_width,cluster_id,hierarchy1_id,hierarchy2_id,hierarchy3_id,hierarchy4_id,hierarchy5_id,sales_x,revenue_x,stock_x,price_x,promo_discount_2_x,sales_y,revenue_y,stock_y,price_y,promo_discount_2_y,sales_x,revenue_x,stock_x,price_x,promo_discount_2_x,sales_y,revenue_y,stock_y,price_y,promo_discount_2_y
0,2.0,13.0,8.0,0,H00,H0001,H000105,H00010510,H0001051017,13562.0,124082.74,981836.0,2.039569e+06,0.0,24087.0,230170.61,2084309.0,3.856664e+06,0.0,431654.0,2553605.029,18607414.0,1.048096e+07,101250.0,6131422.085,2.005676e+07,1.738584e+08,5.788840e+07,1121246.0
1,6.5,13.6,6.5,0,H00,H0001,H000105,H00010510,H0001051006,13562.0,124082.74,981836.0,2.039569e+06,0.0,24087.0,230170.61,2084309.0,3.856664e+06,0.0,431654.0,2553605.029,18607414.0,1.048096e+07,101250.0,6131422.085,2.005676e+07,1.738584e+08,5.788840e+07,1121246.0
2,9.0,12.0,9.0,3,H00,H0001,H000105,H00010510,H0001051017,13562.0,124082.74,981836.0,2.039569e+06,0.0,24087.0,230170.61,2084309.0,3.856664e+06,0.0,431654.0,2553605.029,18607414.0,1.048096e+07,101250.0,6131422.085,2.005676e+07,1.738584e+08,5.788840e+07,1121246.0
3,4.3,13.5,4.3,9,H00,H0001,H000105,H00010510,H0001051017,13562.0,124082.74,981836.0,2.039569e+06,0.0,24087.0,230170.61,2084309.0,3.856664e+06,0.0,431654.0,2553605.029,18607414.0,1.048096e+07,101250.0,6131422.085,2.005676e+07,1.738584e+08,5.788840e+07,1121246.0
4,3.0,10.0,3.0,0,H00,H0001,H000105,H00010500,H0001050001,10525.0,106087.87,1102473.0,1.817095e+06,0.0,24087.0,230170.61,2084309.0,3.856664e+06,0.0,431654.0,2553605.029,18607414.0,1.048096e+07,101250.0,6131422.085,2.005676e+07,1.738584e+08,5.788840e+07,1121246.0


In [422]:
train.shape

(649, 29)

In [423]:
test = q3_submission.merge(sales_by_4,how="inner",on="hierarchy4_id")

In [424]:
test = test.merge(sales_by_3,how="inner",on="hierarchy3_id")

In [425]:
test = test.merge(sales_by_2,how="inner",on="hierarchy2_id")

In [426]:
test = test.merge(sales_by_1,how="inner",on="hierarchy1_id")

In [427]:
test.head()

,product_length,product_depth,product_width,cluster_id,hierarchy1_id,hierarchy2_id,hierarchy3_id,hierarchy4_id,hierarchy5_id,sales_x,revenue_x,stock_x,price_x,promo_discount_2_x,sales_y,revenue_y,stock_y,price_y,promo_discount_2_y,sales_x,revenue_x,stock_x,price_x,promo_discount_2_x,sales_y,revenue_y,stock_y,price_y,promo_discount_2_y
0,5.0,20.0,12.0,NaN,H00,H0004,H000401,H00040105,H0004010534,172314.000,785466.68,4904473.000,1.104469e+06,103475.0,226752.000,1293511.230,6989474.000,3.288612e+06,138200.0,714613.085,4.566959e+06,1.767812e+07,6.348501e+06,247500.0,6131422.085,2.005676e+07,1.738584e+08,5.788840e+07,1121246.0
1,16.5,1.0,8.0,NaN,H00,H0004,H000401,H00040105,H0004010501,172314.000,785466.68,4904473.000,1.104469e+06,103475.0,226752.000,1293511.230,6989474.000,3.288612e+06,138200.0,714613.085,4.566959e+06,1.767812e+07,6.348501e+06,247500.0,6131422.085,2.005676e+07,1.738584e+08,5.788840e+07,1121246.0
2,1.0,1.0,1.0,NaN,H00,H0004,H000403,H00040305,H0004030536,4522.790,181745.16,0.000,2.718356e+05,0.0,18135.907,816004.290,25.744,7.137164e+05,0.0,714613.085,4.566959e+06,1.767812e+07,6.348501e+06,247500.0,6131422.085,2.005676e+07,1.738584e+08,5.788840e+07,1121246.0
3,9.0,6.0,9.0,NaN,H00,H0004,H000402,H00040212,H0004021202,1243.178,20612.65,72556.014,1.013353e+05,0.0,1243.178,20612.650,72556.014,1.013353e+05,0.0,714613.085,4.566959e+06,1.767812e+07,6.348501e+06,247500.0,6131422.085,2.005676e+07,1.738584e+08,5.788840e+07,1121246.0
4,15.0,24.9,9.3,NaN,H00,H0001,H000104,H00010413,H0001041317,2386.000,67415.05,480792.000,2.054066e+06,0.0,35555.000,1075763.739,1071567.000,2.954469e+06,0.0,431654.000,2.553605e+06,1.860741e+07,1.048096e+07,101250.0,6131422.085,2.005676e+07,1.738584e+08,5.788840e+07,1121246.0


In [428]:
test.shape

(49, 29)

In [429]:
len(train.columns)

29

In [430]:
len(test.columns)

29

In [431]:
train.shape

(649, 29)

In [432]:
ph.head()

,product_id,product_length,product_depth,product_width,cluster_id,hierarchy1_id,hierarchy2_id,hierarchy3_id,hierarchy4_id,hierarchy5_id
0,P0000,5.0,20.0,12.0,NaN,H00,H0004,H000401,H00040105,H0004010534
1,P0001,13.5,22.0,20.0,cluster_5,H01,H0105,H010501,H01050100,H0105010006
2,P0002,22.0,40.0,22.0,cluster_0,H03,H0315,H031508,H03150800,H0315080028
3,P0004,2.0,13.0,4.0,cluster_3,H03,H0314,H031405,H03140500,H0314050003
4,P0005,16.0,30.0,16.0,cluster_9,H03,H0312,H031211,H03121109,H0312110917


In [433]:
train.head()

,product_length,product_depth,product_width,cluster_id,hierarchy1_id,hierarchy2_id,hierarchy3_id,hierarchy4_id,hierarchy5_id,sales_x,revenue_x,stock_x,price_x,promo_discount_2_x,sales_y,revenue_y,stock_y,price_y,promo_discount_2_y,sales_x,revenue_x,stock_x,price_x,promo_discount_2_x,sales_y,revenue_y,stock_y,price_y,promo_discount_2_y
0,2.0,13.0,8.0,0,H00,H0001,H000105,H00010510,H0001051017,13562.0,124082.74,981836.0,2.039569e+06,0.0,24087.0,230170.61,2084309.0,3.856664e+06,0.0,431654.0,2553605.029,18607414.0,1.048096e+07,101250.0,6131422.085,2.005676e+07,1.738584e+08,5.788840e+07,1121246.0
1,6.5,13.6,6.5,0,H00,H0001,H000105,H00010510,H0001051006,13562.0,124082.74,981836.0,2.039569e+06,0.0,24087.0,230170.61,2084309.0,3.856664e+06,0.0,431654.0,2553605.029,18607414.0,1.048096e+07,101250.0,6131422.085,2.005676e+07,1.738584e+08,5.788840e+07,1121246.0
2,9.0,12.0,9.0,3,H00,H0001,H000105,H00010510,H0001051017,13562.0,124082.74,981836.0,2.039569e+06,0.0,24087.0,230170.61,2084309.0,3.856664e+06,0.0,431654.0,2553605.029,18607414.0,1.048096e+07,101250.0,6131422.085,2.005676e+07,1.738584e+08,5.788840e+07,1121246.0
3,4.3,13.5,4.3,9,H00,H0001,H000105,H00010510,H0001051017,13562.0,124082.74,981836.0,2.039569e+06,0.0,24087.0,230170.61,2084309.0,3.856664e+06,0.0,431654.0,2553605.029,18607414.0,1.048096e+07,101250.0,6131422.085,2.005676e+07,1.738584e+08,5.788840e+07,1121246.0
4,3.0,10.0,3.0,0,H00,H0001,H000105,H00010500,H0001050001,10525.0,106087.87,1102473.0,1.817095e+06,0.0,24087.0,230170.61,2084309.0,3.856664e+06,0.0,431654.0,2553605.029,18607414.0,1.048096e+07,101250.0,6131422.085,2.005676e+07,1.738584e+08,5.788840e+07,1121246.0


In [434]:
train.shape

(649, 29)

**Label Encoding**

In [435]:
le = preprocessing.LabelEncoder()
le.fit(train.hierarchy5_id)
train.hierarchy5_id = le.transform(train.hierarchy5_id)

In [436]:
le = preprocessing.LabelEncoder()
le.fit(train.hierarchy4_id)
train.hierarchy4_id = le.transform(train.hierarchy4_id)

In [437]:
le = preprocessing.LabelEncoder()
le.fit(train.hierarchy3_id)
train.hierarchy3_id = le.transform(train.hierarchy3_id)

In [438]:
le = preprocessing.LabelEncoder()
le.fit(train.hierarchy2_id)
train.hierarchy2_id = le.transform(train.hierarchy2_id)

In [439]:
le = preprocessing.LabelEncoder()
le.fit(train.hierarchy1_id)
train.hierarchy1_id = le.transform(train.hierarchy1_id)

In [440]:
train.head()

,product_length,product_depth,product_width,cluster_id,hierarchy1_id,hierarchy2_id,hierarchy3_id,hierarchy4_id,hierarchy5_id,sales_x,revenue_x,stock_x,price_x,promo_discount_2_x,sales_y,revenue_y,stock_y,price_y,promo_discount_2_y,sales_x,revenue_x,stock_x,price_x,promo_discount_2_x,sales_y,revenue_y,stock_y,price_y,promo_discount_2_y
0,2.0,13.0,8.0,0,0,1,7,14,27,13562.0,124082.74,981836.0,2.039569e+06,0.0,24087.0,230170.61,2084309.0,3.856664e+06,0.0,431654.0,2553605.029,18607414.0,1.048096e+07,101250.0,6131422.085,2.005676e+07,1.738584e+08,5.788840e+07,1121246.0
1,6.5,13.6,6.5,0,0,1,7,14,26,13562.0,124082.74,981836.0,2.039569e+06,0.0,24087.0,230170.61,2084309.0,3.856664e+06,0.0,431654.0,2553605.029,18607414.0,1.048096e+07,101250.0,6131422.085,2.005676e+07,1.738584e+08,5.788840e+07,1121246.0
2,9.0,12.0,9.0,3,0,1,7,14,27,13562.0,124082.74,981836.0,2.039569e+06,0.0,24087.0,230170.61,2084309.0,3.856664e+06,0.0,431654.0,2553605.029,18607414.0,1.048096e+07,101250.0,6131422.085,2.005676e+07,1.738584e+08,5.788840e+07,1121246.0
3,4.3,13.5,4.3,9,0,1,7,14,27,13562.0,124082.74,981836.0,2.039569e+06,0.0,24087.0,230170.61,2084309.0,3.856664e+06,0.0,431654.0,2553605.029,18607414.0,1.048096e+07,101250.0,6131422.085,2.005676e+07,1.738584e+08,5.788840e+07,1121246.0
4,3.0,10.0,3.0,0,0,1,7,13,23,10525.0,106087.87,1102473.0,1.817095e+06,0.0,24087.0,230170.61,2084309.0,3.856664e+06,0.0,431654.0,2553605.029,18607414.0,1.048096e+07,101250.0,6131422.085,2.005676e+07,1.738584e+08,5.788840e+07,1121246.0


In [441]:
le = preprocessing.LabelEncoder()
le.fit(test.hierarchy5_id)
test.hierarchy5_id = le.transform(test.hierarchy5_id)

In [442]:
le = preprocessing.LabelEncoder()
le.fit(test.hierarchy4_id)
test.hierarchy4_id = le.transform(test.hierarchy4_id)

In [443]:
le = preprocessing.LabelEncoder()
le.fit(test.hierarchy3_id)
test.hierarchy3_id = le.transform(test.hierarchy3_id)

In [444]:
le = preprocessing.LabelEncoder()
le.fit(test.hierarchy2_id)
test.hierarchy2_id = le.transform(test.hierarchy2_id)

In [445]:
le = preprocessing.LabelEncoder()
le.fit(test.hierarchy1_id)
test.hierarchy1_id = le.transform(test.hierarchy1_id)

In [446]:
test.head()

,product_length,product_depth,product_width,cluster_id,hierarchy1_id,hierarchy2_id,hierarchy3_id,hierarchy4_id,hierarchy5_id,sales_x,revenue_x,stock_x,price_x,promo_discount_2_x,sales_y,revenue_y,stock_y,price_y,promo_discount_2_y,sales_x,revenue_x,stock_x,price_x,promo_discount_2_x,sales_y,revenue_y,stock_y,price_y,promo_discount_2_y
0,5.0,20.0,12.0,NaN,0,4,9,11,14,172314.000,785466.68,4904473.000,1.104469e+06,103475.0,226752.000,1293511.230,6989474.000,3.288612e+06,138200.0,714613.085,4.566959e+06,1.767812e+07,6.348501e+06,247500.0,6131422.085,2.005676e+07,1.738584e+08,5.788840e+07,1121246.0
1,16.5,1.0,8.0,NaN,0,4,9,11,13,172314.000,785466.68,4904473.000,1.104469e+06,103475.0,226752.000,1293511.230,6989474.000,3.288612e+06,138200.0,714613.085,4.566959e+06,1.767812e+07,6.348501e+06,247500.0,6131422.085,2.005676e+07,1.738584e+08,5.788840e+07,1121246.0
2,1.0,1.0,1.0,NaN,0,4,11,13,16,4522.790,181745.16,0.000,2.718356e+05,0.0,18135.907,816004.290,25.744,7.137164e+05,0.0,714613.085,4.566959e+06,1.767812e+07,6.348501e+06,247500.0,6131422.085,2.005676e+07,1.738584e+08,5.788840e+07,1121246.0
3,9.0,6.0,9.0,NaN,0,4,10,12,15,1243.178,20612.65,72556.014,1.013353e+05,0.0,1243.178,20612.650,72556.014,1.013353e+05,0.0,714613.085,4.566959e+06,1.767812e+07,6.348501e+06,247500.0,6131422.085,2.005676e+07,1.738584e+08,5.788840e+07,1121246.0
4,15.0,24.9,9.3,NaN,0,1,2,2,3,2386.000,67415.05,480792.000,2.054066e+06,0.0,35555.000,1075763.739,1071567.000,2.954469e+06,0.0,431654.000,2.553605e+06,1.860741e+07,1.048096e+07,101250.0,6131422.085,2.005676e+07,1.738584e+08,5.788840e+07,1121246.0


In [447]:
test.isna().sum()

product_length         2
product_depth          1
product_width          1
cluster_id            49
hierarchy1_id          0
hierarchy2_id          0
hierarchy3_id          0
hierarchy4_id          0
hierarchy5_id          0
sales_x                0
revenue_x              0
stock_x                0
price_x                0
promo_discount_2_x     0
sales_y                0
revenue_y              0
stock_y                0
price_y                0
promo_discount_2_y     0
sales_x                0
revenue_x              0
stock_x                0
price_x                0
promo_discount_2_x     0
sales_y                0
revenue_y              0
stock_y                0
price_y                0
promo_discount_2_y     0
dtype: int64

In [448]:
train["product_length"] =train["product_length"].fillna(train["product_length"].median())

In [478]:
train.product_depth.fillna(0,inplace=True)

In [479]:
train.product_width.fillna(0,inplace=True)

In [480]:
test.product_length.fillna(0,inplace=True)

In [481]:
test.product_depth.fillna(0,inplace=True)

In [482]:
test.product_width.fillna(0,inplace=True)

In [483]:
train.isna().sum()

product_length          0
product_depth         649
product_width          15
cluster_id              0
hierarchy1_id           0
hierarchy2_id           0
hierarchy3_id           0
hierarchy4_id           0
hierarchy5_id           0
sales_x                 0
revenue_x               0
stock_x                 0
price_x                 0
promo_discount_2_x      0
sales_y                 0
revenue_y               0
stock_y                 0
price_y                 0
promo_discount_2_y      0
sales_x                 0
revenue_x               0
stock_x                 0
price_x                 0
promo_discount_2_x      0
sales_y                 0
revenue_y               0
stock_y                 0
price_y                 0
promo_discount_2_y      0
dtype: int64

In [455]:
len(train.columns)

29

In [456]:
len(test.columns)

29

In [457]:
train.shape

(649, 29)

In [458]:
test.shape

(49, 29)

In [459]:
train.tail()

,product_length,product_depth,product_width,cluster_id,hierarchy1_id,hierarchy2_id,hierarchy3_id,hierarchy4_id,hierarchy5_id,sales_x,revenue_x,stock_x,price_x,promo_discount_2_x,sales_y,revenue_y,stock_y,price_y,promo_discount_2_y,sales_x,revenue_x,stock_x,price_x,promo_discount_2_x,sales_y,revenue_y,stock_y,price_y,promo_discount_2_y
644,5.0,20.0,15.0,0,2,10,44,96,198,5272.420,13519.52,86646.141,19568.25,0.0,62167.858,316404.57,298976.391,144261.92,0.0,62385.818,318739.28,303421.82,145666.37,0.0,76225.737,429366.41,503602.324,230860.14,0.0
645,5.0,NaN,NaN,9,2,10,44,96,199,5272.420,13519.52,86646.141,19568.25,0.0,62167.858,316404.57,298976.391,144261.92,0.0,62385.818,318739.28,303421.82,145666.37,0.0,76225.737,429366.41,503602.324,230860.14,0.0
646,5.0,NaN,NaN,9,2,10,44,96,199,5272.420,13519.52,86646.141,19568.25,0.0,62167.858,316404.57,298976.391,144261.92,0.0,62385.818,318739.28,303421.82,145666.37,0.0,76225.737,429366.41,503602.324,230860.14,0.0
647,0.0,0.0,0.0,7,2,10,44,97,200,56895.438,302885.05,212330.250,124693.67,0.0,62167.858,316404.57,298976.391,144261.92,0.0,62385.818,318739.28,303421.82,145666.37,0.0,76225.737,429366.41,503602.324,230860.14,0.0
648,5.0,NaN,NaN,9,2,10,44,97,201,56895.438,302885.05,212330.250,124693.67,0.0,62167.858,316404.57,298976.391,144261.92,0.0,62385.818,318739.28,303421.82,145666.37,0.0,76225.737,429366.41,503602.324,230860.14,0.0


In [460]:
test.tail()

,product_length,product_depth,product_width,cluster_id,hierarchy1_id,hierarchy2_id,hierarchy3_id,hierarchy4_id,hierarchy5_id,sales_x,revenue_x,stock_x,price_x,promo_discount_2_x,sales_y,revenue_y,stock_y,price_y,promo_discount_2_y,sales_x,revenue_x,stock_x,price_x,promo_discount_2_x,sales_y,revenue_y,stock_y,price_y,promo_discount_2_y
44,5.5,22.5,19.0,NaN,3,13,27,35,41,3610.000,87087.12,1459195.000,4.415950e+06,0.0,18885.000,579544.32,3917367.000,2.451781e+07,0.0,554623.000,2.616701e+06,2.391910e+07,4.077430e+07,15640.0,876207.000,5669057.14,5.871648e+07,1.126469e+08,132214.0
45,6.0,32.5,23.0,NaN,3,13,27,34,40,1956.000,123388.35,452034.000,8.243961e+06,0.0,18885.000,579544.32,3917367.000,2.451781e+07,0.0,554623.000,2.616701e+06,2.391910e+07,4.077430e+07,15640.0,876207.000,5669057.14,5.871648e+07,1.126469e+08,132214.0
46,9.0,28.0,20.0,NaN,3,13,28,36,42,73709.000,601834.50,2080837.000,2.117366e+06,9900.0,90400.000,740626.72,3001945.000,4.765174e+06,15640.0,554623.000,2.616701e+06,2.391910e+07,4.077430e+07,15640.0,876207.000,5669057.14,5.871648e+07,1.126469e+08,132214.0
47,5.0,20.0,30.0,NaN,2,10,20,27,33,217.960,2334.71,4445.429,1.404450e+03,0.0,217.960,2334.71,4445.429,1.404450e+03,0.0,62385.818,3.187393e+05,3.034218e+05,1.456664e+05,0.0,76225.737,429366.41,5.036023e+05,2.308601e+05,0.0
48,NaN,NaN,NaN,NaN,2,9,19,26,32,11699.919,94627.25,83656.504,3.617440e+04,0.0,11699.919,94627.25,83656.504,3.617440e+04,0.0,13839.919,1.106271e+05,2.001805e+05,8.519377e+04,0.0,76225.737,429366.41,5.036023e+05,2.308601e+05,0.0


In [461]:
# Missing of the entry can also be a valuable information
# So we will create a column that is False when value is missing
# We encoded missingness in categorical columns so we will just create _na columns for numerical types
def fix_missing(df, col, name):
    if is_numeric_dtype(col):
        if pd.isnull(col).sum():
            df[name+"_na"] = pd.isnull(col)
        df[name] = col.fillna(col.median())
        
# We will have codes starting from 0 (for missing)
# def numericalize(df, col, name):
#     if not is_numeric_dtype(col):
#         print(df[name].cat.categories)
#         print(col.cat.codes+1)
#         df[name] = col.cat.codes+1
        
# dic = dict()
# def numericalize(df, col, name):
#     if not is_numeric_dtype(col):
#         if name == 'hierarchy4_id':
#             for i in range(len(df[name].cat.categories)):
#                 dic[str(df[name].cat.categories[i])] = list(set(df[name].cat.codes))[i]+1
#         df[name] = col.cat.codes+1

    
def proc_df(df, y_fld):
    
#     y = df[y_fld].values
#     df.drop([y_fld], axis = 1, inplace = True)
    
    for n, c in df.items():
        fix_missing(df, c, n)
        
#     for n, c in df.items():
#         numericalize(df, c, n)
    
    y = df[y_fld].values
    df.drop([y_fld], axis = 1, inplace = True)
    
    res = [df, y]
    
    return res

### Model

In [348]:
X_train = train.drop("cluster_id",axis=1)[:500]
y_train = train.cluster_id[:500].values
X_test = test.drop("cluster_id",axis=1)
y_test = test.cluster_id.values

In [354]:
# Fitting Random Forest Classification to the Training set
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 42)
classifier.fit(X_train, y_train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [288]:
rfc_pred = classifier.predict(X_test)

In [297]:
print ("Random Forest Train Accuracy Baseline:", metrics.accuracy_score(y_train, classifier.predict(scaled_X_train)))
print ("Random Forest Test Accuracy Baseline:", metrics.accuracy_score(y_test, classifier.predict(scaled_X_test)))
#print('Accuracy of random forest classifier on test set: {:.2f}'.format(rfc.score(X_test, y_test)))

Random Forest Train Accuracy Baseline: 0.852
Random Forest Test Accuracy Baseline: 0.6845637583892618


In [ ]:
# Feature Scaling
scaler = StandardScaler()
scaled_X_train = scaler.fit_transform(X_train)
scaled_X_test = scaler.transform(X_test)

# Fitting Random Forest Classification to the Training set
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 42)
classifier.fit(scaled_X_train, y_train)

rfc_pred = classifier.predict(X_test)

print ("Random Forest Train Accuracy Baseline:", metrics.accuracy_score(y_train, classifier.predict(scaled_X_train)))
print ("Random Forest Test Accuracy Baseline:", metrics.accuracy_score(y_test, classifier.predict(scaled_X_test)))
#print('Accuracy of random forest classifier on test set: {:.2f}'.format(rfc.score(X_test, y_test)))

**Dividing the datasets**

In [289]:
train.merge(ph,on="hierarchy5_id",how="inner").head()

,product_length_x,product_depth_x,product_width_x,cluster_id_x,hierarchy1_id_x,hierarchy2_id_x,hierarchy3_id_x,hierarchy4_id_x,hierarchy5_id,sales_x,revenue_x,stock_x,price_x,promo_discount_2_x,sales_y,revenue_y,stock_y,price_y,promo_discount_2_y,sales_x,revenue_x,stock_x,price_x,promo_discount_2_x,sales_y,revenue_y,stock_y,price_y,promo_discount_2_y,product_id,product_length_y,product_depth_y,product_width_y,cluster_id_y,hierarchy1_id_y,hierarchy2_id_y,hierarchy3_id_y,hierarchy4_id_y
0,2.0,13.0,8.0,0,H00,H0001,H000105,H00010510,H0001051017,13562.0,124082.74,981836.0,2.039569e+06,0.0,24087.0,230170.61,2084309.0,3.856664e+06,0.0,431654.0,2553605.029,18607414.0,1.048096e+07,101250.0,6131422.085,2.005676e+07,1.738584e+08,5.788840e+07,1121246.0,P0346,9.0,12.0,9.0,cluster_3,H00,H0001,H000105,H00010510
1,2.0,13.0,8.0,0,H00,H0001,H000105,H00010510,H0001051017,13562.0,124082.74,981836.0,2.039569e+06,0.0,24087.0,230170.61,2084309.0,3.856664e+06,0.0,431654.0,2553605.029,18607414.0,1.048096e+07,101250.0,6131422.085,2.005676e+07,1.738584e+08,5.788840e+07,1121246.0,P0376,2.0,13.0,8.0,cluster_0,H00,H0001,H000105,H00010510
2,2.0,13.0,8.0,0,H00,H0001,H000105,H00010510,H0001051017,13562.0,124082.74,981836.0,2.039569e+06,0.0,24087.0,230170.61,2084309.0,3.856664e+06,0.0,431654.0,2553605.029,18607414.0,1.048096e+07,101250.0,6131422.085,2.005676e+07,1.738584e+08,5.788840e+07,1121246.0,P0707,4.3,13.5,4.3,cluster_9,H00,H0001,H000105,H00010510
3,9.0,12.0,9.0,3,H00,H0001,H000105,H00010510,H0001051017,13562.0,124082.74,981836.0,2.039569e+06,0.0,24087.0,230170.61,2084309.0,3.856664e+06,0.0,431654.0,2553605.029,18607414.0,1.048096e+07,101250.0,6131422.085,2.005676e+07,1.738584e+08,5.788840e+07,1121246.0,P0346,9.0,12.0,9.0,cluster_3,H00,H0001,H000105,H00010510
4,9.0,12.0,9.0,3,H00,H0001,H000105,H00010510,H0001051017,13562.0,124082.74,981836.0,2.039569e+06,0.0,24087.0,230170.61,2084309.0,3.856664e+06,0.0,431654.0,2553605.029,18607414.0,1.048096e+07,101250.0,6131422.085,2.005676e+07,1.738584e+08,5.788840e+07,1121246.0,P0376,2.0,13.0,8.0,cluster_0,H00,H0001,H000105,H00010510


In [183]:
train = train.drop(train.columns[30:],axis=1)

In [184]:
test = test.merge(ph,on="hierarchy5_id",how="inner")

In [185]:
test = test.drop(test.columns[30:],axis=1)

In [186]:
test.head()

,product_length_x,product_depth_x,product_width_x,cluster_id_x,hierarchy1_id_x,hierarchy2_id_x,hierarchy3_id_x,hierarchy4_id_x,hierarchy5_id,sales_x,revenue_x,stock_x,price_x,promo_discount_2_x,sales_y,revenue_y,stock_y,price_y,promo_discount_2_y,sales_x,revenue_x,stock_x,price_x,promo_discount_2_x,sales_y,revenue_y,stock_y,price_y,promo_discount_2_y,product_id
0,5.0,20.0,12.0,NaN,H00,H0004,H000401,H00040105,H0004010534,172314.00,785466.68,4904473.0,1.104469e+06,103475.0,226752.000,1293511.23,6989474.000,3.288612e+06,138200.0,714613.085,4.566959e+06,1.767812e+07,6.348501e+06,247500.0,6131422.085,2.005676e+07,1.738584e+08,5.788840e+07,1121246.0,P0000
1,16.5,1.0,8.0,NaN,H00,H0004,H000401,H00040105,H0004010501,172314.00,785466.68,4904473.0,1.104469e+06,103475.0,226752.000,1293511.23,6989474.000,3.288612e+06,138200.0,714613.085,4.566959e+06,1.767812e+07,6.348501e+06,247500.0,6131422.085,2.005676e+07,1.738584e+08,5.788840e+07,1121246.0,P0189
2,16.5,1.0,8.0,NaN,H00,H0004,H000401,H00040105,H0004010501,172314.00,785466.68,4904473.0,1.104469e+06,103475.0,226752.000,1293511.23,6989474.000,3.288612e+06,138200.0,714613.085,4.566959e+06,1.767812e+07,6.348501e+06,247500.0,6131422.085,2.005676e+07,1.738584e+08,5.788840e+07,1121246.0,P0410
3,16.5,1.0,8.0,NaN,H00,H0004,H000401,H00040105,H0004010501,172314.00,785466.68,4904473.0,1.104469e+06,103475.0,226752.000,1293511.23,6989474.000,3.288612e+06,138200.0,714613.085,4.566959e+06,1.767812e+07,6.348501e+06,247500.0,6131422.085,2.005676e+07,1.738584e+08,5.788840e+07,1121246.0,P0455
4,1.0,1.0,1.0,NaN,H00,H0004,H000403,H00040305,H0004030536,4522.79,181745.16,0.0,2.718356e+05,0.0,18135.907,816004.29,25.744,7.137164e+05,0.0,714613.085,4.566959e+06,1.767812e+07,6.348501e+06,247500.0,6131422.085,2.005676e+07,1.738584e+08,5.788840e+07,1121246.0,P0023
